<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/6_performance_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 🔹 Step 1: Load Dataset
dataset_path = r"C:\LCTSF\Dataset\timeseries_dataset.csv"  # Dataset location
df = pd.read_csv(dataset_path)

# 🔹 Step 2: Data Preprocessing
data = df['value'].values.reshape(-1, 1)  # Extract time-series values
scaler = MinMaxScaler(feature_range=(0, 1))  # Normalize data
data_scaled = scaler.fit_transform(data)

# 🔹 Step 3: Create Sequences for Forecasting
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, 0])  # Input sequence
        y.append(data[i+seq_length, 0])    # Target output
    return np.array(X), np.array(y)

sequence_length = 50  # Time steps in sequence
X, y = create_sequences(data_scaled, sequence_length)

# Split into Training & Testing Sets
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Reshape data for LSTM input
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 🔹 Step 4: Define a Baseline Model (Traditional LSTM)
baseline_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=False),
    tf.keras.layers.Dense(1)
])

# Compile & Train Baseline Model
baseline_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
baseline_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 🔹 Step 5: Define the Primal-Dual Model (Constrained Learning)
class PrimalDualLoss(tf.keras.losses.Loss):
    """
    Implements the Primal-Dual Optimization Loss:
    - Adds a constraint penalty term to standard loss.
    - Uses Lagrange multipliers to dynamically adjust penalties.
    """
    def __init__(self, lambda_dual=0.1, max_error=0.05):
        super(PrimalDualLoss, self).__init__()
        self.lambda_dual = tf.Variable(lambda_dual, trainable=False, dtype=tf.float32)
        self.max_error = max_error

    def call(self, y_true, y_pred):
        error = tf.abs(y_true - y_pred)
        primal_loss = tf.reduce_mean(tf.square(error))
        dual_penalty = tf.reduce_mean(tf.maximum(error - self.max_error, 0) ** 2)
        return primal_loss + self.lambda_dual * dual_penalty

# Define Constrained Model
constrained_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True, input_shape=(sequence_length, 1)),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=False),
    tf.keras.layers.Dense(1)
])

# Compile & Train Constrained Model
constrained_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=PrimalDualLoss())
constrained_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 🔹 Step 6: Evaluate Model Performance
y_pred_baseline = baseline_model.predict(X_test)
y_pred_constrained = constrained_model.predict(X_test)

# Rescale Predictions to Original Scale
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_baseline_rescaled = scaler.inverse_transform(y_pred_baseline)
y_pred_constrained_rescaled = scaler.inverse_transform(y_pred_constrained)

# Compute Performance Metrics
mae_baseline = mean_absolute_error(y_test_rescaled, y_pred_baseline_rescaled)
mae_constrained = mean_absolute_error(y_test_rescaled, y_pred_constrained_rescaled)

rmse_baseline = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_baseline_rescaled))
rmse_constrained = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_constrained_rescaled))

# Print Results
print(f"🔹 Baseline Model - MAE: {mae_baseline:.4f}, RMSE: {rmse_baseline:.4f}")
print(f"🔹 Constrained Model - MAE: {mae_constrained:.4f}, RMSE: {rmse_constrained:.4f}")

# 🔹 Step 7: Visualize Model Performance
plt.figure(figsize=(12, 5))

# Plot Baseline vs Constrained Model Predictions
plt.plot(y_test_rescaled, label="Actual Values", color="blue")
plt.plot(y_pred_baseline_rescaled, label="Baseline Predictions", color="red", linestyle="dashed")
plt.plot(y_pred_constrained_rescaled, label="Constrained Predictions", color="green", linestyle="dotted")

plt.title("Performance Comparison: Baseline vs Primal-Dual Model")
plt.xlabel("Time Steps")
plt.ylabel("Value")
plt.legend()
plt.show()

# 🔹 Step 8: Analyze Error Distribution
error_baseline = np.abs(y_test_rescaled - y_pred_baseline_rescaled)
error_constrained = np.abs(y_test_rescaled - y_pred_constrained_rescaled)

plt.figure(figsize=(10, 5))
plt.hist(error_baseline, bins=30, alpha=0.5, label="Baseline Error", color="red")
plt.hist(error_constrained, bins=30, alpha=0.5, label="Constrained Model Error", color="green")

plt.title("Error Distribution: Baseline vs Constrained Model")
plt.xlabel("Absolute


SyntaxError: unterminated string literal (detected at line 121) (<ipython-input-1-03c819689e17>, line 121)